In [2]:
from data import dataset as ds

ModuleNotFoundError: No module named 'torchvison'

In [1]:
ds = ds.get_CIFAR_10()

NameError: name 'ds' is not defined

### Evaluating the Model on the Validation Set
Now that the model is trained, let's evaluate its performance on the validation (test) set to see how well it generalizes to unseen data.

NameError: name 'model' is not defined